In [96]:
import pandas as pd
import numpy as np
import seaborn as sns

PATH_READ = 'raw_train.csv'
PATH_WRITE = 'train_with_features_v2.csv'
df = pd.read_csv(PATH_READ, index_col='Timestamp')

Preprocessing, cleaning unwanted cols

In [97]:
df = df.drop(['InstrumentID','TradingDay','PreClosePrice','UpdateTime'], axis=1)
df.index = pd.to_datetime(df.index)

print(df.columns)

Index(['PreClosePrice', 'Open', 'High', 'Low', 'Volume', 'Turnover', 'Close'], dtype='object')


Add hist return features

In [98]:
def hist_ret_close(df, param):
    return df['Close'] / df['Close'].shift(param)

df.insert(loc=df.columns.get_loc('Close')+1, column= 'Hist_Ret_Close_15min', value= hist_ret_close(df,15))
df.insert(loc=df.columns.get_loc('Close')+1, column= 'Hist_Ret_Close_10min', value= hist_ret_close(df,10))
df.insert(loc=df.columns.get_loc('Close')+1, column= 'Hist_Ret_Close_5min', value= hist_ret_close(df,5))
df.insert(loc=df.columns.get_loc('Close')+1, column= 'Hist_Ret_Close_3min', value= hist_ret_close(df,3))

print(df.columns)

Index(['PreClosePrice', 'Open', 'High', 'Low', 'Volume', 'Turnover', 'Close',
       'Hist_Ret_Close_3min', 'Hist_Ret_Close_5min', 'Hist_Ret_Close_10min',
       'Hist_Ret_Close_15min'],
      dtype='object')


Add log return features

In [99]:
df['Log_Return_Close_1min'] = np.log(df['Close'] / df['Close'].shift(1))

print(df.columns)
print(df['Log_Return_Close_1min'])

Index(['PreClosePrice', 'Open', 'High', 'Low', 'Volume', 'Turnover', 'Close',
       'Hist_Ret_Close_3min', 'Hist_Ret_Close_5min', 'Hist_Ret_Close_10min',
       'Hist_Ret_Close_15min', 'Log_Return_Close_1min'],
      dtype='object')
Timestamp
2023-06-21 09:26:00         NaN
2023-06-21 09:31:00   -0.015623
2023-06-21 09:32:00    0.024979
2023-06-21 09:33:00    0.007725
2023-06-21 09:34:00   -0.007563
                         ...   
2023-06-27 14:58:00   -0.001032
2023-06-27 14:59:00    0.000000
2023-06-27 15:00:00    0.000000
2023-06-27 15:01:00    0.001754
2023-06-27 15:35:00    0.000000
Name: Log_Return_Close_1min, Length: 729, dtype: float64


Add average price and mean price from 4

In [100]:
df['Avg Price'] = df['Turnover']/df['Volume']

# For unchanged prices, fill zero
df['Avg Price'].fillna(0, inplace=True)


df['4mean Price'] = (df['High']+df['Low']+df['Close']+df['Open'])/4.0


Add log return based on avg price

In [101]:
df['Log_Return_Avg_1min'] = np.log(df['Avg Price'] / df['Avg Price'].shift(1))

print(df.columns)
print(df['Log_Return_Avg_1min'])

Index(['PreClosePrice', 'Open', 'High', 'Low', 'Volume', 'Turnover', 'Close',
       'Hist_Ret_Close_3min', 'Hist_Ret_Close_5min', 'Hist_Ret_Close_10min',
       'Hist_Ret_Close_15min', 'Log_Return_Close_1min', 'Avg Price',
       '4mean Price', 'Log_Return_Avg_1min'],
      dtype='object')
Timestamp
2023-06-21 09:26:00         NaN
2023-06-21 09:31:00   -0.004298
2023-06-21 09:32:00    0.005441
2023-06-21 09:33:00    0.015268
2023-06-21 09:34:00   -0.003514
                         ...   
2023-06-27 14:58:00    0.001320
2023-06-27 14:59:00         NaN
2023-06-27 15:00:00         NaN
2023-06-27 15:01:00         NaN
2023-06-27 15:35:00         NaN
Name: Log_Return_Avg_1min, Length: 729, dtype: float64


C:\Users\Aaron\anaconda3\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Hist based on avg price

In [102]:
def hist_ret_avg(df, param):
    return df['Avg Price'] / df['Avg Price'].shift(param)

df['Hist_Ret_Avg_3min'] = hist_ret_avg(df, 3)
df['Hist_Ret_Avg_5min'] = hist_ret_avg(df, 5)
df['Hist_Ret_Avg_10min'] = hist_ret_avg(df, 10)
df['Hist_Ret_Avg_15min'] = hist_ret_avg(df, 15)

Add targets based on close price

In [103]:
df['Target_Close_1min'] = (df['Close'].shift(-1) - df['Close'])/df['Close']

df['Target_Close_5min_Mean'] = (df['Close'].rolling(window=5).mean().shift(-5) - df['Close']) / df['Close']

df['Target_Close_10min_Mean'] = (df['Close'].rolling(window=10).mean().shift(-10) - df['Close']) / df['Close']

df['Target_Close_15min_Mean'] = (df['Close'].rolling(window=10).mean().shift(-15) - df['Close']) / df['Close']

Add targets based on avg price

In [104]:
df['Target_Avg_1min'] = (df['Avg Price'].shift(-1) - df['Avg Price'])/df['Avg Price']

df['Target_Avg_5min_Mean'] = (df['Avg Price'].rolling(window=5).mean().shift(-5) - df['Avg Price']) / df['Avg Price']

df['Target_Avg_10min_Mean'] = (df['Avg Price'].rolling(window=10).mean().shift(-10) - df['Avg Price']) / df['Avg Price']

df['Target_Avg_15min_Mean'] = (df['Avg Price'].rolling(window=10).mean().shift(-15) - df['Avg Price']) / df['Avg Price']

Experiment with rolling volatility

In [105]:
def rolling_std(df, window_size):
    return df['Close'].rolling(window=window_size).std()

df['Vol_5min_Close'] = rolling_std(df, 5)
df['Vol_10min_Close'] = rolling_std(df, 10)
df['Vol_15min_Close'] = rolling_std(df, 15)

print(df.columns)

Index(['PreClosePrice', 'Open', 'High', 'Low', 'Volume', 'Turnover', 'Close',
       'Hist_Ret_Close_3min', 'Hist_Ret_Close_5min', 'Hist_Ret_Close_10min',
       'Hist_Ret_Close_15min', 'Log_Return_Close_1min', 'Avg Price',
       '4mean Price', 'Log_Return_Avg_1min', 'Hist_Ret_Avg_3min',
       'Hist_Ret_Avg_5min', 'Hist_Ret_Avg_10min', 'Hist_Ret_Avg_15min',
       'Target_Close_1min', 'Target_Close_5min_Mean',
       'Target_Close_10min_Mean', 'Target_Close_15min_Mean', 'Target_Avg_1min',
       'Target_Avg_5min_Mean', 'Target_Avg_10min_Mean',
       'Target_Avg_15min_Mean', 'Vol_5min_Close', 'Vol_10min_Close',
       'Vol_15min_Close'],
      dtype='object')


Sharpe on close price

In [106]:
risk_free_rate = 0

df['Sharpe_Log_ret_1min/Vol_5min_Close'] = (df['Log_Return_Close_1min'] - risk_free_rate) / df['Vol_5min_Close']
df['Sharpe_Log_ret_1min/Vol_10min_Close'] = (df['Log_Return_Close_1min'] - risk_free_rate) / df['Vol_10min_Close']
df['Sharpe_Log_ret_1min/Vol_15min_Close'] = (df['Log_Return_Close_1min'] - risk_free_rate) / df['Vol_15min_Close']


Cleaning NaN and unwanted data

In [107]:
print(df.shape)

df = df[(df['Target_Close_1min'] != 0)
                      & (df['Target_Close_5min_Mean'] != 0)
                      & (df['Target_Close_10min_Mean'] != 0)
                      & (df['Target_Close_15min_Mean'] != 0)
                     ]
pd.set_option ('mode.use_inf_as_na', True)
df = df.dropna()

print(df)

(729, 33)
                     PreClosePrice     Open     High      Low  Volume  \
Timestamp                                                               
2023-06-21 09:45:00          320.8  325.416  325.416  324.244   44726   
2023-06-21 09:46:00          320.8  324.200  324.200  323.000   37280   
2023-06-21 09:47:00          320.8  324.025  324.025  323.033   25290   
2023-06-21 09:48:00          320.8  322.933  322.933  321.000   47670   
2023-06-21 09:49:00          320.8  321.819  323.653  321.809   38780   
...                            ...      ...      ...      ...     ...   
2023-06-27 14:43:00          318.9  319.878  320.880  319.780   28910   
2023-06-27 14:44:00          318.9  320.491  321.132  320.400   45330   
2023-06-27 14:45:00          318.9  321.400  322.229  321.307   76080   
2023-06-27 14:46:00          318.9  321.971  321.971  321.100   29700   
2023-06-27 14:47:00          318.9  321.299  321.400  320.800   25350   

                        Turnover    Clos

Saving CSV

In [108]:
df.to_csv(PATH_WRITE)